In [1]:
from qiskit import QuantumCircuit, execute, transpile
from dell_runtime import DellRuntimeProvider

In [2]:
provider = DellRuntimeProvider()

In [3]:
provider.remote('http://qdr-dev.oro-sandbox-small1.k8s.cec.lab.emc.com')

Opening webpage https://appsso.login.scfd.isus.emc.com/oauth/authorize?response_type=code&client_id=5c731039-4384-4ea1-b134-c9c9c8e25131&redirect_uri=http%3A%2F%2Fqdr-dev.oro-sandbox-small1.k8s.cec.lab.emc.com%2Fcallback&scope=openid+roles+user_attributes&state=logJg1n1BJ4jTOinXyvXsL74iS7FDW



In [4]:
provider.runtime.programs()

{'5a487b2140c8': <qiskit.providers.ibmq.runtime.runtime_program.RuntimeProgram at 0x7ff8c44e6ee0>}

In [5]:
RUNTIME_PROGRAM = """
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):

    user_messenger.publish({'results': 'intermittently'})

    circuits = transpile(
        circuits,
    )

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    for i in range(0, 10):
        result = backend.run(circuits, **kwargs).result()
        user_messenger.publish({f'result_{i}': result})
    
    user_messenger.publish('completed', final=True)
    print("job complete successfully")
"""

In [6]:
RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

In [7]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)

In [8]:
program_id

'5e49102b984b'

In [9]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [17]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs, callback=print)

In [18]:
job.get_unread_messages()

[]

In [19]:
job.result(timeout=20)

Exception: Timeout while waiting for job 72422b87bca5.

In [ ]:
job.status()